In [ ]:
import sys
#sys.path.append('/uio/kant/geo-metos-u7/franzihe/Documents/research_Haukeli/Python/windrose')
sys.path.append('/Volumes/SANDISK128/Documents/Research_UiO_50/Python/windrose/')
from windrose import WindroseAxes
import netCDF4
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#sys.path.append('/uio/kant/geo-metos-u7/franzihe/Documents/Thesis/Python')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
import createFolder as cF
import math
#%matplotlib inline

In [ ]:
from matplotlib import colors

import matplotlib.cm as cm

In [ ]:
station = 'Haukeliseter'

In [ ]:
txt_dir      = '../../Data/Eklima/'+station
txt_filename = 'obs_'+str(2016)+'_'+str(2017)

In [ ]:
Haukeli = pd.read_csv(txt_dir+'/'+txt_filename+'.txt', sep =';', header=19, skipfooter = 2, engine = 'python')

In [ ]:
Year = Haukeli['Year']
Month = Haukeli['Mnth']
Date = Haukeli['Date']
Time = Haukeli['Time(NMT)']
obsvar = Haukeli['RR_1'].astype(float)         # Precipitation (last 1 hour); [mm]
obsvar_TA = Haukeli['TA'].astype(float)

obsvar_DD = Haukeli['DD'].astype(float)             # Wind direction (FF); [degrees] Generell vindretning siste 10 minutter 
obsvar_FF = Haukeli['FF'].astype(float)             # Wind speed (10 meters above ground);m/s

In [ ]:
# create array of values we want to have (precip, Wind)
arr = (np.vstack((Year,Month,Date,Time,obsvar,obsvar_DD,obsvar_FF,obsvar_TA)).T)
idx = np.where(arr[:,:] == -9999)
arr[idx] = np.nan

In [ ]:
def get_netCDF_variable(f, var_name, var, dim):
    v_0m = f.createVariable(varname=var_name, datatype=var.dtype, dimensions=dim, zlib=True)
    v_0m[:] = var[:]
    return(v_0m)

In [ ]:
txt_dir
for i in range(int(arr.shape[0]/24)-1):
    if int(arr[24+i*24,2]) < 10:
        dd = '0%s' %str(int(arr[24+i*24,2]))
    else:
        dd = str(int(arr[24+i*24,2]))
        
    if int(arr[24+i*24,1]) < 10:
        mm = '0%s' %str(int(arr[24+i*24,1]))
    else:
        mm = str(int(arr[24+i*24,1]))
    filename = 'precip_wind_temp_'+str(int(arr[24+i*24,0]))+mm+dd+'.nc'
    ### write netCDF file
    fw = netCDF4.Dataset(txt_dir+'/'+filename,'w')
    ### create dimensions
    fw.createDimension('time', arr[24+i*24:24+i*24+24,:].shape[0])
    fw.createDimension('YYYYMMDD_HH', arr[24+i*24:24+i*24+24,0:4].shape[1])

    time_w      = get_netCDF_variable(fw, 'time_stamp', arr[24+i*24:24+i*24+24,0:4], ('time','YYYYMMDD_HH' ))
    precip_w    = get_netCDF_variable(fw, 'RR_1',       arr[24+i*24:24+i*24+24,4], ('time',) )
    winddir_w   = get_netCDF_variable(fw, 'DD',         arr[24+i*24:24+i*24+24,5], ('time',))
    windspeed_w = get_netCDF_variable(fw, 'FF',         arr[24+i*24:24+i*24+24,6], ('time',))
    air_temp_w  = get_netCDF_variable(fw, 'TA',         arr[24+i*24:24+i*24+24,7], ('time',))

    fw.close()